### Setting Envirionment

In [ ]:
! pip install pandas
! pip install openai
! pip install langchain

In [ ]:
import os
import json
import openai
import pandas as pd
import csv
import random
import string

In [ ]:
with open('openai_api_key.txt', 'r') as f:
    openai.api_key = f.read().strip()
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ.get("OPENAI_API_KEY") # api_key 확인 필요할 때

In [ ]:
# Calling prompts
with open('sprompt_int.txt', 'r') as f:
    sprompt_int = f.read()

with open('sprompt_obj.txt', 'r') as f:
    sprompt_obj = f.read()

with open('sprompt_plt.txt', 'r') as f:
    sprompt_plt = f.read()

In [ ]:
# Checking the prompt
print(sprompt_int[50:120])
print(sprompt_obj[50:120])
print(sprompt_plt[50:120])

### Scene Genaration with Langchain Schema

- [cls0] OBJECT UNCERTAINTY

In [ ]:
# OBJECT UNCERTAINTY
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model="gpt-4-0613", temperature=0.7)
output = []

for _ in range(1):
    messages = [
        SystemMessage(content=f"'{sprompt_obj}'"),
        HumanMessage(content="Write sentences each about a situation with an ordinary object that is out of context."),
        ]

    scene = chat(messages)
    scene_list = [scene.content] # Etracting only the content of the AIMessage as dict
    
    output.append(scene_list)
    output_list = ';'.join([','.join(lst) for lst in output])

    # 세미콜론 기준으로 문자열 분리하고 리스트 생성
split_output_lists = [s.strip().split("',") for s in output_list.split(';')]

# Flatten list 로 변환 후 cls 0 DataFrame 생성
flat_list = [item[0] for item in split_output_lists]
df0 = pd.DataFrame({
    'scn_sentence': flat_list,
    'scn_cls': [0] * len(flat_list)
})

In [ ]:
df0

- [cls1] INTENTION UNCERTAINTY

In [ ]:
# INTENTION UNCERTAINTY
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
output = []

for _ in range(1):
    messages = [
        SystemMessage(content=f"'{sprompt_int}'"),
        HumanMessage(content="Write short sentences each of which is about a realistic situation that gives a question mark about a character's behavior."),
        ]

    scene = chat(messages)
    scene_list = [scene.content] # Etracting only the content of the AIMessage as dict
    
    output.append(scene_list)
    output_list = ';'.join([','.join(lst) for lst in output])

    # 세미콜론 기준으로 문자열 분리하고 리스트 생성
split_output_lists = [s.strip().split("',") for s in output_list.split(';')]

# Flatten list 로 변환 후 cls 1 DataFrame 생성
flat_list = [item[0] for item in split_output_lists]
df1 = pd.DataFrame({
    'scn_sentence': flat_list,
    'scn_cls': [1] * len(flat_list)
})

In [ ]:
df1

- cls[2] EVENT UNCERTAINTY

In [ ]:
# EVENT UNCERTAINTY
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model="gpt-4-0613", temperature=1)
output = []

for _ in range(1):
    messages = [
        SystemMessage(content=f"'{sprompt_plt}'"),
        HumanMessage(content="Write sentences that depict a noticeable change in a common scenario involving a person or object."),
        ]

    scene = chat(messages)
    scene_list = [scene.content] # Etracting only the content of the AIMessage as dict
    
    output.append(scene_list)
    output_list = ';'.join([','.join(lst) for lst in output])

    # 세미콜론 기준으로 문자열 분리하고 리스트 생성
split_output_lists = [s.strip().split("',") for s in output_list.split(';')]

# Flatten list 로 변환 후 cls 2 DataFrame 생성
flat_list = [item[0] for item in split_output_lists]
df2 = pd.DataFrame({
    'scn_sentence': flat_list,
    'scn_cls': [2] * len(flat_list)
})

In [ ]:
df2

In [ ]:
# 세 데이터프레임을 row-wise로 합침
df = pd.concat([df0, df1, df2], axis=0)

In [ ]:
df

### 무작위로 생성된 ID를 기준으로 정렬한 최종 df 생성

In [ ]:
# 숫자새 개와 알파벳을 섞어서 고유한 난수를 생성
def generate_random_id():
    numbers = ''.join(random.sample(string.digits, 3))  # 3개의 숫자 선택
    letters = ''.join(random.sample(string.ascii_uppercase, 3))  # 3개의 대문자 알파벳 선택
    # 알파벳과 숫자를 합친 후 무작위로 섞는다.
    random_id = ''.join(random.sample(numbers + letters, 6))
    return random_id

# 중복되지 않는 난수를 생성
unique_ids = set()
while len(unique_ids) < len(df):
    unique_id = generate_random_id()
    if unique_id not in unique_ids:
        unique_ids.add(unique_id)

# 각 문장에 대한 난수 ID를 DataFrame 가장 왼쪽에 새로운 열을 삽입하는 방식으로 추가
df.insert(0, 'scn_id', list(unique_ids))

In [ ]:
# 'scn_id'를 기준으로 오름차순 정렬 (해도 되고 안 해도 되고)
df = df.sort_values(by='scn_id')

### Saving files

In [ ]:
# Set the name of output file
FileName = '0904_add_75_kdh'

In [ ]:
# Save DataFrame to CSV
df.to_csv(f'_output_scene/{FileName}.csv', index=False)